In [1]:
'''
Objective: This code initializes required libraries for the Carla simulator to add a vehicle, and attach LiDAR and depth sensors to it. The libraries facilitate the simulation and visualization of the vehicle and sensor data.

Libraries Overview:
    - carla: Provides the interface to interact with the Carla simulator, allowing vehicle and sensor manipulation.
    - math: Supplies mathematical functions (e.g., for positioning, rotations).
    - random: Generates random numbers, useful for randomizing vehicle or sensor placement.
    - time: Provides time control (e.g., for pauses and timing).
    - numpy (np): Enables numerical operations on arrays, important for sensor data processing.
    - cv2: Handles image processing and display (for camera sensor data).
    - open3d as o3d: Used to visualize 3D point cloud data, essential for rendering LiDAR data.
    - matplotlib and cm: Used for color mapping and data visualization (e.g., for depth sensor).
'''
import carla 
import math
import random
import time
import numpy as np
import cv2
import open3d as o3d # to mention
from matplotlib import cm
import matplotlib

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
'''
Objective: Establish a connection to the Carla simulator and access the simulation world.

1. carla.Client('localhost', 2000):
    - Purpose: Creates a client to connect to the Carla simulator running on localhost at port 2000.
    - Usage: Acts as the main interface to communicate with the Carla server.

2. client.get_world():
    - Purpose: Retrieves the current simulation environment or "world" in Carla.
    - Usage: Provides access to all elements in the simulation, such as vehicles, sensors, and actors, allowing manipulation within the world.
'''
client = carla.Client('localhost', 2000)
world = client.get_world()

In [4]:
'''
Objective: Retrieve available spawn points within the Carla simulation world.

1.world.get_map():
    - Purpose: Fetches the current map of the simulation world.
    - Usage: Provides information related to the simulation environment's layout (e.g., roads, intersections).

2.get_spawn_points():
    - Purpose: Returns a list of predefined spawn points (positions and orientations) on the map where vehicles can be placed.
    - Usage: Used to randomly or specifically select a location to spawn vehicles.
'''
spawn_points = world.get_map().get_spawn_points()

In [5]:
'''
Objective: Access the blueprint library to retrieve vehicle and sensor definitions.

1. world.get_blueprint_library():
    - Purpose: Fetches the blueprint library, which contains definitions for various actors, including vehicles, sensors, and props.
    - Usage: Enables selection and instantiation of specific vehicle or sensor types within the simulation, allowing customization and flexibility in vehicle configurations.
'''
bp_lib = world.get_blueprint_library()

In [6]:
'''
Objective: Spawn a specific vehicle in the Carla simulation.

1. `bp_lib.find('vehicle.tesla.cybertruck')`:
   - Purpose: Locates the blueprint for the Tesla Cybertruck within the blueprint library.
   - Usage: Retrieves the vehicle's specifications to allow for instantiation in the simulation.

2. `world.try_spawn_actor(vehicle_bp, spawn_points[0])`:
   - Purpose: Attempts to spawn an actor (the Tesla Cybertruck) at the first available spawn point.
   - Usage: Creates an instance of the vehicle in the simulation; if spawning is successful, it returns the vehicle object, allowing further interaction.
'''

vehicle_bp = bp_lib.find('vehicle.tesla.cybertruck')
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])

In [7]:
'''
Objective: Define the initial camera position for the simulation.

1. `CAMERA_POS_Z = 3`:
   - Purpose: Sets the Z-axis position of the camera above the vehicle.
   - Usage: Specifies how high the camera will be placed relative to the vehicle, enhancing the field of view.

2. `CAMERA_POS_X = 0`:
   - Purpose: Sets the X-axis position of the camera.
   - Usage: Defines the lateral position of the camera relative to the vehicle, allowing for adjustments in viewpoint.

3. `camera_init_trans = carla.Transform(carla.Location(z = CAMERA_POS_Z, x = CAMERA_POS_X))`:
   - Purpose: Creates a transformation object representing the camera's initial position.
   - Usage: Combines the defined X and Z positions into a transform that can be used when attaching the camera to the vehicle.
'''
CAMERA_POS_Z = 3
CAMERA_POS_X = 0
camera_init_trans = carla.Transform(carla.Location(z = CAMERA_POS_Z, x = CAMERA_POS_X))

In [8]:
'''
Objective: Spawn an RGB camera sensor and attach it to the vehicle.

1. `camera_bp = bp_lib.find('sensor.camera.rgb')`:
   - Purpose: Locates the blueprint for the RGB camera within the blueprint library.
   - Usage: Retrieves the specifications needed to instantiate the camera in the simulation.

2. `camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)`:
   - Purpose: Spawns the RGB camera actor at the specified initial transform and attaches it to the vehicle.
   - Usage: Ensures that the camera moves with the vehicle, allowing for real-time image capture during the simulation.
'''
camera_bp = bp_lib.find('sensor.camera.rgb')
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

In [9]:
'''
Objective: Process and store incoming camera images.

1. `def camera_callback(image, data_dict):`
   - Purpose: Defines a callback function that processes raw image data from the camera.
   - Parameters:
     - `image`: The raw image data received from the camera sensor.
     - `data_dict`: A dictionary where the processed image will be stored.

2. Inside the function:
   - `data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))`:
     - Purpose: Copies the raw image data and reshapes it into a 4-channel image (including an alpha channel).
     - `np.copy(image.raw_data)`: Creates a copy of the raw image data to prevent modifications to the original data.
     - `np.reshape(..., (image.height, image.width, 4))`: Reshapes the data into a format suitable for further processing, where `image.height` and `image.width` correspond to the image's dimensions.

3. Usage: This function allows real-time updates of the camera image data for visualization or analysis.

'''
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [10]:
'''
1. `depth_camera_bp = bp_lib.find('sensor.camera.depth')`:
   - Purpose: Locates the blueprint for the depth camera within the blueprint library.
   - Usage: Retrieves the specifications needed to instantiate the depth camera in the simulation.

2. `depth_camera = world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to=vehicle)`:
   - Purpose: Spawns the depth camera actor at the specified initial transform and attaches it to the vehicle.
   - Usage: Ensures that the depth camera moves with the vehicle, capturing depth data during the simulation.
'''
depth_camera_bp = bp_lib.find('sensor.camera.depth')
depth_camera = world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to = vehicle)

In [11]:
'''
Objective: Define a callback function to process depth camera images.

1. `def depth_callback(image, data_dict):`:
   - Purpose: Defines a function that is triggered when a depth image is received from the camera.
   - Usage: Processes the incoming depth data and stores it in a dictionary for further analysis.

2. `image.convert(carla.ColorConverter.LogarithmicDepth)`:
   - Purpose: Converts the raw depth image data to a logarithmic scale.
   - Usage: Enhances the visibility of depth information, making it easier to interpret.

3. `data_dict['depth_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))`:
   - Purpose: Copies and reshapes the processed raw depth data into a NumPy array.
   - Usage: Converts the raw depth data into a format suitable for manipulation, matching the camera's resolution and including an alpha channel.
'''

def depth_callback(image, data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth)
    data_dict['depth_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [12]:
'''
Objective: Set up color maps and add a 3D axis to the Open3D visualizer.

1. `VIRIDIS = np.array(matplotlib.colormaps.get_cmap('plasma').colors)`:
   - Purpose: Retrieves the colors from the 'plasma' colormap and stores them as a NumPy array.
   - Usage: Provides a set of colors for visualizing depth data or other metrics in the simulation.

2. `VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])`:
   - Purpose: Generates a linearly spaced array ranging from 0.0 to 1.0, with the same number of elements as the colors in the 'plasma' colormap.
   - Usage: Serves as a range for mapping values to the colormap.

3. `COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])`:
   - Purpose: Creates a similar range for the 'winter' colormap.
   - Usage: Provides another set of colors for visualizing different data.

4. `COOL = np.array(matplotlib.colormaps.get_cmap('winter')(COOL_RANGE))`:
   - Purpose: Retrieves colors from the 'winter' colormap and stores them as a NumPy array.
   - Usage: Offers a contrasting color palette for visualization.

5. `COOL = COOL[:,:3]`:
   - Purpose: Trims the 'winter' colormap to exclude the alpha channel, retaining only RGB values.
   - Usage: Ensures compatibility with visualization libraries that require RGB input.

6. `def add_open3d_axis(vis):`:
   - Purpose: Defines a function to add a small 3D axis to the Open3D visualizer.
   - Usage: Enhances the visual context by providing reference axes.

7. Inside the function:
   - **Creating a LineSet**: Sets up a 3D line set for the axes.
   - **Defining Points and Lines**: Specifies the points and lines for the X, Y, and Z axes.
   - **Setting Colors**: Colors the axes red, green, and blue for X, Y, and Z respectively.
   - **Adding Geometry**: Adds the axis geometry to the Open3D visualizer.

This setup enhances visualization by adding reference axes and establishing color maps for depth data interpretation.
'''

VIRIDIS = np.array(matplotlib.colormaps.get_cmap('plasma').colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])

COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL = np.array(matplotlib.colormaps.get_cmap('winter')(COOL_RANGE))
COOL = COOL[:,:3]

def add_open3d_axis(vis):
    ''' Add a small 3D axis on Open3D visualizer '''
    axis = o3d.geometry.LineSet()
    axis.points = o3d.utility.Vector3dVector(np.array([
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    axis.lines = o3d.utility.Vector2iVector(np.array([
        [0, 1],
        [0, 2],
        [0, 3]]))
    axis.colors = o3d.utility.Vector3dVector(np.array([
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    vis.add_geometry(axis)

In [14]:
'''
Objective: Configure and spawn a LiDAR sensor, attaching it to the vehicle.

1. `lidar_bp = bp_lib.find('sensor.lidar.ray_cast')`:
   - Purpose: Locates the blueprint for the ray-cast LiDAR sensor within the blueprint library.
   - Usage: Retrieves the specifications necessary for instantiating the LiDAR sensor in the simulation.

2. `lidar_bp.set_attribute('range', '100.0')`:
   - Purpose: Sets the maximum range of the LiDAR sensor to 100 meters.
   - Usage: Defines how far the LiDAR can detect points in the environment.

3. `lidar_bp.set_attribute('noise_stddev', '0.1')`:
   - Purpose: Configures the standard deviation of noise added to the LiDAR measurements.
   - Usage: Simulates real-world inaccuracies, making the sensor data more realistic.

4. `lidar_bp.set_attribute('upper_fov', '15.0')`:
   - Purpose: Sets the upper field of view (FOV) of the LiDAR sensor to 15 degrees.
   - Usage: Determines the vertical range of detection for the sensor.

5. `lidar_bp.set_attribute('lower_fov', '-25.0')`:
   - Purpose: Sets the lower field of view (FOV) of the LiDAR sensor to -25 degrees.
   - Usage: Extends the vertical range of detection below the vehicle.

6. `lidar_bp.set_attribute('channels', '64.0')`:
   - Purpose: Configures the number of channels for the LiDAR sensor to 64.
   - Usage: Affects the resolution and detail of the point cloud data collected.

7. `lidar_bp.set_attribute('rotation_frequency', '20.0')`:
   - Purpose: Sets the rotation frequency of the LiDAR sensor to 20 Hz.
   - Usage: Determines how quickly the LiDAR scans its surroundings.

8. `lidar_bp.set_attribute('points_per_second', '500000')`:
   - Purpose: Configures the LiDAR to emit 500,000 points per second.
   - Usage: Influences the density of the point cloud data collected.

9. `lidar_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))`:
   - Purpose: Creates a transformation object for the LiDAR's initial position.
   - Usage: Sets the LiDAR's location above the vehicle using previously defined coordinates.

10. `lidar = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=vehicle)`:
    - Purpose: Spawns the LiDAR actor at the specified transform and attaches it to the vehicle.
    - Usage: Ensures that the LiDAR moves with the vehicle, collecting point cloud data in real-time during the simulation.


'''

lidar_bp = bp_lib.find('sensor.lidar.ray_cast')
lidar_bp.set_attribute('range', '100.0')
lidar_bp.set_attribute('noise_stddev', '0.1')
lidar_bp.set_attribute('upper_fov', '15.0')
lidar_bp.set_attribute('lower_fov', '-25.0')
lidar_bp.set_attribute('channels', '64.0')
lidar_bp.set_attribute('rotation_frequency', '20.0')
lidar_bp.set_attribute('points_per_second', '500000')

lidar_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))
lidar = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to = vehicle)

In [13]:
'''
Objective: Process LiDAR point cloud data and prepare it for visualization in Open3D.

1. `def lidar_callback(point_cloud, point_list):`:
   - Purpose: Defines a function to handle incoming LiDAR point cloud data.
   - Usage: Prepares the data for visualization by converting it into a suitable format for Open3D.

2. `data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))`:
   - Purpose: Copies the raw point cloud data from the LiDAR sensor into a NumPy array.
   - Usage: Converts the binary data into a format that can be reshaped and manipulated.

3. `data = np.reshape(data, (int(data.shape[0] / 4), 4))`:
   - Purpose: Reshapes the data array into a shape suitable for processing, with each point represented by four values (X, Y, Z, intensity).
   - Usage: Prepares the data for further analysis by isolating the point coordinates and intensity.

4. `intensity = data[:, -1]`:
   - Purpose: Extracts the intensity values from the last column of the reshaped data.
   - Usage: Used to compute a corresponding color for each point based on its intensity.

5. `intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))`:
   - Purpose: Normalizes the intensity values to a color scale.
   - Usage: Converts intensity values into a range suitable for color mapping.

6. `int_color = np.c_[...]`:
   - Purpose: Interpolates the intensity colors using the predefined `VID_RANGE` and `VIRIDIS` colormap.
   - Usage: Creates RGB color values corresponding to each point based on its intensity.

7. `points = data[:, :-1]`:
   - Purpose: Extracts the X, Y, and Z coordinates from the point data.
   - Usage: Prepares the coordinates for visualization.

8. `points[:, :1] = -points[:, :1]`:
   - Purpose: Inverts the X-axis coordinates.
   - Usage: Adjusts the coordinate system as needed for visualization.

9. `point_list.points = o3d.utility.Vector3dVector(points)`:
   - Purpose: Sets the points of the Open3D point cloud object.
   - Usage: Prepares the points for rendering in the visualizer.

10. `point_list.colors = o3d.utility.Vector3dVector(int_color)`:
    - Purpose: Sets the colors of the points in the Open3D point cloud object.
    - Usage: Applies the interpolated colors to the corresponding points for visualization.
'''
def lidar_callback(point_cloud, point_list):
    ''' Prepares a point cloud with intensity colors ready to be consumed by Open3D'''
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype = np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))

    # Isolate the intensity and compute a color for it
    intensity = data[:, -1]
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_[
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 0]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 1]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 2])]

    points = data[:, :-1]
    points[:, :1] = -points[:, :1]

    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)

In [15]:
'''
Objective: Initialize a point cloud object for visualization.

1. `point_list = o3d.geometry.PointCloud()`:
   - Purpose: Creates a new instance of a PointCloud object in Open3D.
   - Usage: This object will hold and manage the 3D point data collected from the LiDAR sensor, enabling visualization and manipulation of the point cloud in the Open3D environment.

'''
point_list = o3d.geometry.PointCloud()

In [16]:
''' 
Objective: Retrieve the dimensions of the camera's image.

1. `image_w = camera_bp.get_attribute("image_size_x").as_int()`:
   - Purpose: Fetches the width of the camera's image in pixels.
   - Usage: Stores the image width, which can be used for processing or displaying the captured images.

2. `image_h = camera_bp.get_attribute("image_size_y").as_int()`:
   - Purpose: Fetches the height of the camera's image in pixels.
   - Usage: Stores the image height, ensuring that the dimensions are available for image processing and visualization tasks.

'''
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()

In [17]:
'''
Objective: Initialize a dictionary to store camera and depth image data.

1. `camera_data = {'image': np.zeros((image_h, image_w, 4)), 'depth_image': np.zeros((image_h, image_w, 4))}`:
   - Purpose: Creates a dictionary to hold two types of image data: RGB and depth images.
   - Usage: Initializes both images as zero-filled NumPy arrays with dimensions corresponding to the camera's height and width, including an alpha channel. This structure allows for easy storage and access of image data during the simulation
'''
camera_data = {'image' : np.zeros((image_h, image_w, 4)), 'depth_image': np.zeros((image_h, image_w, 4))}

In [18]:
'''
Objective: Set up a callback function to process incoming camera images.

1. `camera.listen(lambda image: camera_callback(image, camera_data))`:
   - Purpose: Registers a callback function to be called whenever a new image is received from the camera sensor.
   - Parameters:
     - `lambda image`: A lambda function that takes the incoming image as input.
     - `camera_callback(image, camera_data)`: Calls the `camera_callback` function, passing the incoming image and a dictionary to store the processed image data.
   - Usage: Allows for real-time processing and storage of camera images, enabling further analysis or visualization.

'''
camera.listen(lambda image: camera_callback(image, camera_data))

In [19]:
'''
Objective: Set up the LiDAR sensor to listen for incoming point cloud data.

1. `lidar.listen(lambda data: lidar_callback(data, point_list))`:
   - Purpose: Registers a callback function (`lidar_callback`) that will be triggered whenever point cloud data is received from the LiDAR sensor.
   - Usage: This lambda function passes the incoming data and the `point_list` object to the `lidar_callback`, facilitating real-time processing and visualization of the LiDAR point cloud as it is generated during the simulation.

'''
lidar.listen(lambda data: lidar_callback(data, point_list))

In [20]:
''' 
Objective: Set up the depth camera to listen for incoming depth images.

1. `depth_camera.listen(lambda image: depth_callback(image, camera_data))`:
   - Purpose: Registers a callback function (`depth_callback`) that will be triggered whenever a depth image is captured by the depth camera.
   - Usage: This lambda function passes the captured depth image and the `camera_data` dictionary to the `depth_callback`, enabling real-time processing of depth data as it becomes available during the simulation.

'''

depth_camera.listen(lambda image: depth_callback(image, camera_data))

In [21]:
'''
Objective: Enable the vehicle's autopilot mode.

1. `vehicle.set_autopilot(True)`:
   - Purpose: Activates the autopilot feature for the vehicle.
   - Usage: Allows the vehicle to autonomously navigate the environment, following traffic rules and pre-defined routes in the Carla simulation, simulating the behavior of self-driving cars.

'''
vehicle.set_autopilot(True)

In [ ]:
'''
Objective: Set up visualization windows and render LiDAR data while displaying RGB and depth images in real time.

1. `cv2.namedWindow('RGB Camera', cv2.WINDOW_NORMAL)`:
   - Purpose: Creates a window named "RGB Camera" for displaying RGB images.
   - Usage: The `cv2.WINDOW_NORMAL` flag allows the window to be resized.

2. `cv2.namedWindow('Depth Camera', cv2.WINDOW_NORMAL)`:
   - Purpose: Creates a window named "Depth Camera" for displaying depth images.
   - Usage: Similar to the RGB window, this window can also be resized.

3. `vis = o3d.visualization.Visualizer()`:
   - Purpose: Initializes a visualizer object from Open3D for rendering point cloud data.
   - Usage: Prepares the visualizer for displaying 3D data.

4. `vis.create_window(...)`:
   - Purpose: Creates a new window for visualization.
   - Parameters:
     - `window_name`: Title of the window.
     - `width` and `height`: Dimensions of the window.
     - `left` and `top`: Position of the window on the screen.
   - Usage: Sets up the visual environment for rendering.

5. `vis.get_render_option().background_color = [0.05, 0.05, 0.05]`:
   - Purpose: Changes the background color of the visualizer window.
   - Usage: Provides a dark background to enhance the visibility of the point cloud.

6. `vis.get_render_option().point_size = 1`:
   - Purpose: Sets the size of the points in the point cloud visualization.
   - Usage: Adjusts how the points are rendered for better visibility.

7. `vis.get_render_option().show_coordinate_frame = True`:
   - Purpose: Displays a coordinate frame in the visualizer.
   - Usage: Helps to orient the viewer in 3D space.

8. `add_open3d_axis(vis)`:
   - Purpose: Calls a function to add a 3D axis to the visualizer.
   - Usage: Enhances the context of the visualization with reference axes.

9. `frame = 0`:
   - Purpose: Initializes a frame counter for controlling the visualization loop.

10. `while True:`:
    - Purpose: Starts an infinite loop for real-time visualization and image display.

11. Inside the loop:
    - **Adding Geometry**:
      - `if frame == 2: vis.add_geometry(point_list)`: Adds the point cloud geometry to the visualizer on the third iteration.
    - **Updating Geometry**:
      - `vis.update_geometry(point_list)`: Updates the point cloud data in the visualizer.
    - **Polling Events**:
      - `vis.poll_events()`: Processes events such as window resizing or closing.
    - **Updating Renderer**:
      - `vis.update_renderer()`: Renders the current frame.
    - **Frame Delay**:
      - `time.sleep(0.005)`: Introduces a short delay to control the loop speed.
    - **Displaying Images**:
      - `cv2.imshow('RGB Camera', camera_data['image'])`: Shows the RGB image in the RGB Camera window.
      - `cv2.imshow('Depth Camera', camera_data['depth_image'])`: Shows the depth image in the Depth Camera window.
    - **Exit Condition**:
      - `if cv2.waitKey(1) == ord('q'):`: Exits the loop when 'q' is pressed.

12. **Cleanup**:
    - `cv2.destroyAllWindows()`: Closes all OpenCV windows.
    - Stops and destroys all sensors and the vehicle:
      - `lidar.stop()`, `lidar.destroy()`, `camera.stop()`, `camera.destroy()`, `depth_camera.stop()`, `depth_camera.destroy()`.
    - Destroys all vehicle and sensor actors in the simulation world:
      - `for actor in world.get_actors().filter('*vehicle*'): actor.destroy()`.
      - `for sensor in world.get_actors().filter('*sensor*'): sensor.destroy()`.

'''

cv2.namedWindow('RGB Camera', cv2.WINDOW_NORMAL)
cv2.namedWindow('Depth Camera', cv2.WINDOW_NORMAL)

vis = o3d.visualization.Visualizer()
vis.create_window(
    window_name = 'Carla Lidar',
    width = 960,
    height = 540,
    left = 480,
    top = 270)

vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().point_size = 1
vis.get_render_option().show_coordinate_frame = True
add_open3d_axis(vis)
frame = 0

while True:
    if frame == 2:
        vis.add_geometry(point_list)
    vis.update_geometry(point_list)
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.005)
    frame +=1
    
    cv2.imshow('RGB Camera', camera_data['image'])
    cv2.imshow('Depth Camera', camera_data['depth_image'])
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()
lidar.stop()
lidar.destroy()
camera.stop()
camera.destroy()
depth_camera.stop()
depth_camera.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()